In [1]:
!pip install -q openai #--progress-bar off

In [49]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [ ]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = "xxxx" # "" #input("INSERT YOU OPENAI API KEY:")
# clear_output()

In [45]:
output_datapath = "data/output/2023.11.15_Llama2_7B_API/"

In [16]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [18]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [22]:
data = pd.read_excel("../data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [27]:
template_part1 = """[INST]
<<SYS>>
In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The text to be processed is:
<</SYS>>

'''
"""

template_part2 = """
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [93]:
data_sel = data #.sample(2)

In [94]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [95]:
import time

cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    query = template_part1 + text + template_part2
    
    body = {
      "model": "meta-llama/Llama-2-7b-chat-hf",
      "messages": [
          {"role": "system", "content": "You are a helpful assistant."}, 
          {"role": "user", "content": query}],
      "temperature": 0.0
    }
    
    result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
    
    print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    
    with open(output_datapath+f'sdg_classification_Llama2_7B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    cnt+=1
    time.sleep(0.5)

0 6668 0   ---SDG 3---
1 6099 0   ---SDG 8---
2 6471 0   ---SDG 1---

The text processed can be classified under SDG 1, which is "No Poverty". The text mentions that the convict, Afroz Khan Pathan, spent 17 years in jail and completed his BA and MA degrees, a Yoga course, and conducted de-radicalisation sessions in prison. This suggests that he was poor and marginalized, and that he was able to access education and rehabilitation programs while in prison. Additionally, the text notes that Pathan was convicted under various provisions of the Unlawful Activities (Prevention) Act, which suggests that he was involved in illegal activities related to terrorism, which can be linked to SDG 16, "Peace, Justice and Strong Institutions".
3 1786 0   ---SDG 4---
4 2143 0   ---SDG 7: Affordable and Clean Energy---
5 3377 0   ---SDG 3---
6 5589 0   ---SDG 3---

The text you provided is primarily related to the gaming industry and the release of digital-only games, including Alan Wake 2. While there 

145 869 2   ---SDG 2: Zero Hunger---
146 3205 2   ---SDG 2: Zero Hunger---
147 6506 2   ---SDG 2---
148 1622 2   ---SDG 2---
149 2827 2   ---SDG 7: Affordable and Clean Energy---
150 223 2   ---SDG 2---
151 1319 2   ---SDG 2---
152 5700 2   ---SDG 2---
153 977 2   ---SDG 8---
154 3039 2   ---SDG 6---
155 6419 2   ---SDG 2---
156 5694 2   ---SDG 8---
157 4433 2   ---SDG 3---
158 654 2   ---SDG 2: Zero Hunger---
159 4178 2   ---SDG 12: Responsible Consumption and Production---
160 5812 2   ---SDG 2---
161 5130 2   ---SDG 12: Responsible Consumption and Production---
162 4078 2   ---SDG 0---
163 646 2   ---SDG 2---
164 4120 2   ---SDG 2---
165 894 2   ---SDG 2: Zero Hunger---
166 1114 2   ---SDG 7: Affordable and Clean Energy---
167 211 2   ---SDG 12: Responsible Consumption and Production---
168 4039 2   ---SDG 12: Responsible Consumption and Production---
169 4054 2   ---SDG 9---
170 1005 2   ---SDG 1---
171 471 2   ---SDG 0---
172 362 2   ---SDG 8: Decent Work and Economic Growth---
17

426 3138 7   ---SDG 7: Affordable and Clean Energy---
427 6251 7   ---SDG 7: Affordable and Clean Energy---
428 6074 7   ---SDG 7: Affordable and Clean Energy---
429 5346 7   ---SDG 7: Affordable and Clean Energy---
430 6226 7   ---SDG 7---
431 3417 7   ---SDG 7: Affordable and Clean Energy---
432 3016 7   ---SDG 7: Affordable and Clean Energy---
433 6798 7   ---SDG 7: Affordable and Clean Energy---
434 4893 7   ---SDG 7: Affordable and Clean Energy---
435 4955 7   ---SDG 7---
436 2617 7   ---SDG 7: Affordable and Clean Energy---
437 4275 7   ---SDG 7: Affordable and Clean Energy---
438 5874 7   ---SDG 7---
439 3068 7   ---SDG 12---
440 1780 7   ---SDG 7: Affordable and Clean Energy---
441 1273 7   ---SDG 7: Affordable and Clean Energy---
442 4795 7   ---SDG 7: Affordable and Clean Energy---
443 3256 7   ---SDG 7---
444 2876 7   ---SDG 7: Affordable and Clean Energy---
445 751 7   ---SDG 7: Affordable and Clean Energy---
446 2893 7   ---SDG 7---
447 2329 7   ---SDG 7: Affordable and Cl

642 2303 10   ---SDG 10---
643 6552 10   ---SDG 10---
644 1119 10   ---SDG 8---
645 570 10   ---SDG 8: Decent Work and Economic Growth---
646 5043 10   ---SDG 17---
647 334 10   ---SDG 0---
648 3153 10   ---SDG 8: Decent Work and Economic Growth---
649 365 10   ---SDG 10---
650 3782 10   ---SDG 16---
651 5986 10   ---SDG 8---
652 1908 10   ---SDG 5---
653 3420 10   ---SDG 10---
654 2233 10   ---SDG 8: Decent Work and Economic Growth---
655 6504 10   ---SDG 16---
656 4945 10   ---SDG 10---
657 1329 10   ---SDG 8---
658 6782 10   ---SDG 8---
659 406 10   ---SDG 8: Decent Work and Economic Growth---
660 1968 11   ---SDG 3---
661 1902 11   ---SDG 11---
662 6209 11   ---SDG 7---
663 2986 11   ---SDG 11---
664 3871 11   ---SDG 8---
665 1564 11   ---SDG 11---
666 4187 11   ---SDG 11---
667 4353 11   ---SDG 8: Promote Economic Growth and Job Creation---
668 4349 11   ---SDG 11---
669 3479 11   ---SDG 11---
670 1989 11   ---SDG 16---
671 739 11   ---SDG 11---
672 1381 11   ---SDG 4---
673 3418 

882 6698 14   ---SDG 12---
883 1348 14   ---SDG 12---
884 4396 14   ---SDG 12---
885 3059 14   ---SDG 6---
886 2693 14   ---SDG 14: Life Below Water---
887 5734 14   ---SDG 12---
888 1089 14   ---SDG 12: Responsible Consumption and Production---
889 5290 14   ---SDG 14: Life Below Water---
890 5720 14   ---SDG 12---
891 204 14   ---SDG 12---
892 6541 14   ---SDG 12---
893 2042 14   ---SDG 16---
894 3687 14   ---SDG 2---
895 5519 14   ---SDG 14: Life Below Water---
896 3637 14   ---SDG 12---
897 4240 14   ---SDG 8---
898 640 14   ---SDG 12---
899 3336 14   ---SDG 12---
900 893 15   ---SDG 15: Life On Land---
901 6606 15   ---SDG 7---
902 4075 15   ---SDG 15---
903 3920 15   ---SDG 15---
904 794 15   ---SDG 8---
905 78 15   ---SDG 11---
906 2788 15   ---SDG 15---
907 3261 15   ---SDG 15---
908 3358 15   ---SDG 15---
909 759 15   ---SDG 15: Life On Land---
910 1455 15   ---SDG 15---
911 724 15   ---SDG 15---
912 1167 15   ---SDG 6---
913 5970 15   ---SDG 17---
914 4268 15   ---SDG 15---
9

In [96]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [97]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    if len(res[3]['choices'])>1:
        raise Exception('ERROR: Multilabel detected!')
    sdg_pred = res[3]['choices'][0]['message']['content']
    res4df = (res[0], res[1], res[2], sdg_pred)
    print(res4df)
    tuple_list.append(res4df)

(167, 211, 2, '  ---SDG 12: Responsible Consumption and Production---')
(933, 3170, 15, '  ---SDG 12: Responsible Consumption and Production---')
(653, 3420, 10, '  ---SDG 10---')
(519, 4081, 8, '  ---SDG 8: Promote Economic Growth and Job Creation---')
(90, 5668, 1, '  ---SDG 10: Reduced Inequalities---')
(533, 6656, 8, '  ---SDG 12: Responsible Consumption and Production---')
(483, 3317, 8, '  ---SDG 7: Affordable and Clean Energy---')
(753, 3777, 12, '  ---SDG 12: Responsible Consumption and Production---')
(52, 6243, 0, "  ---SDG 4---\n\nThe text you provided can be classified under Sustainable Development Goal 4: Quality Education. The Union Minister's statement about Article 44 of the Constitution and the implementation of a Uniform Civil Code aligns with this goal, as it relates to the provision of education and the protection of marginalized communities.")
(340, 5775, 5, '  ---SDG 5: Gender Equality---')
(211, 2539, 3, '  ---SDG 7: Affordable and Clean Energy---')
(345, 6627, 5

In [98]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [99]:
results_processed

,cnt,id,true_label,predicted_label
0,167,211,2,---SDG 12: Responsible Consumption and Produ...
1,933,3170,15,---SDG 12: Responsible Consumption and Produ...
2,653,3420,10,---SDG 10---
3,519,4081,8,---SDG 8: Promote Economic Growth and Job Cr...
4,90,5668,1,---SDG 10: Reduced Inequalities---
...,...,...,...,...
1015,771,1616,12,---SDG 12---
1016,314,1094,5,---SDG 4---
1017,133,98,2,---SDG 2---
1018,441,1273,7,---SDG 7: Affordable and Clean Energy---


In [102]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,---SDG 3---
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 1---\n\nThe text processed can be cla...
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 4---
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 7: Affordable and Clean Energy---
...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 16---
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 6---
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---


In [103]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,---SDG 3---,3
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,---SDG 8---,8
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 1---\n\nThe text processed can be cla...,1
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,---SDG 4---,4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,---SDG 7: Affordable and Clean Energy---,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,---SDG 16---,16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,---SDG 16---,16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,---SDG 6---,6
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,---SDG 16---,16


In [107]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,44,39,44,44,39,39,39,44,44,44,44
1,24,23,24,24,23,23,23,24,24,24,24
10,28,28,28,28,28,28,28,28,28,28,28
11,52,46,52,52,46,46,46,52,52,52,52
12,106,105,106,106,105,105,105,106,106,106,106
13,27,27,27,27,27,27,27,27,27,27,27
14,18,18,18,18,18,18,18,18,18,18,18
15,38,38,38,38,38,38,38,38,38,38,38
16,52,51,52,52,51,51,51,52,52,52,52


In [109]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [110]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.11      0.08      0.10        60
           1       0.79      0.32      0.45        60
           2       0.92      0.38      0.54        60
           3       0.56      0.95      0.70        60
           4       0.68      0.85      0.76        60
           5       0.96      0.72      0.82        60
           6       0.53      0.85      0.65        60
           7       0.49      0.93      0.64        60
           8       0.27      0.57      0.37        60
           9       0.81      0.35      0.49        60
          10       0.71      0.33      0.45        60
          11       0.62      0.53      0.57        60
          12       0.36      0.63      0.46        60
          13       0.85      0.38      0.53        60
          14       0.94      0.28      0.44        60
          15       0.89      0.57      0.69        60
          16       0.65      0.57      0.61        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
